# Ecuación de calor

Tenemos que la ecuación de calor es una de las ecuaciones diferenciales en derivadas parciales más importantes y tiene la siguiente forma
\begin{equation}
    \frac{\partial T}{\partial t} = \alpha \left(\frac{\partial^2T}{\partial x^2} + \frac{\partial^2T}{\partial y^2}\right)
\end{equation}
donde $T(x,y,t)$ es una variable escalar que se puede relacionar con la temperatura y depende tanto de las coordenadas cartesianas como del tiempo. Para el caso de este proyecto, se considera la ecuación en dos dimensiones. Aunado a esto y para poder resolverla, se necesitan tanto condiciones iniciales como condiciones de frontera. Las condiciones iniciales las denotamos como $T(x,y,0)$ mientras que las condiciones de frontera varían dependiendo de la figura propuesta.

### Metodología numérica

Para poder resolver estos tipos de ecuaciones diferenciales, se hacen uso de diversos métodos numéricos. La ecuación de calor es de tipo parabólica, que son las que se emplean para caracterizar problemas variables en el tiempo. El método empleado para la resolución de esta ecuación es el de diferencias finitas. Las ecuaciones parabólicas pueden resolverse sustituyendo las diferencias finitas por las derivadas parciales. Sin embargo, también tenemos que considerar los cambios en el tiempo además de los cambios en el espacio. Estas ecuaciones parabólicas están abiertas temporalmente; esto quiere decir que técnicamente la variable temporal puede seguir "corriendo" de manera indefinida. Debido a su naturaleza dependiente del tiempo, las soluciones a estas ecuaciones implican una serie de retos, notablemente la estabilidad.

La ecuación de calor requiere una aproximación de la primera derivada en el tiempo y de la segunda derivada en el espacio para cada coordenada. Utilizando diferencias finitas, tenemos la posibilidad de escribir las derivadas parciales de la ecuación de calor como aproximaciones. Contamos con varios índices que son realmente útiles a la hora de programar y resolver de manera numérica. Los índices $i$ y $j$ son para las posiciones espaciales en $x$ y $y$ respectivamente y $l$ es el índice relacionado con el tiempo.

La aproximación para la derivada temporal la podemos escribir de la siguiente manera
\begin{equation*}
\frac{\partial T}{\partial t} \approx \frac{T_{i,j}^{l+1} - T_{i,j}^{l}}{\Delta t}
\end{equation*}
donde $\Delta t$ es el paso que se toma en el tiempo. Mientras más pequeños sean esos pasos, mejor será la aproximación. Esta expresión tiene un error de $o(\Delta t)$, lo cual es proporcional al tamaño del paso en el tiempo. Notamos que la suma se lleva a cabo en el índice $l$, ya que este corresponde al índice asociado con el tiempo.

Tenemos además que la segunda derivada con respecto a $x$ se aproxima a
\begin{equation*}
\frac{\partial^2 T}{\partial x^2} \approx \frac{T_{i+1,j}^{l} - 2T_{i,j}^{l} + T_{i-1,j}^{l}}{(\Delta x)^2}
\end{equation*}
y la aproximación para la segunda derivada con respecto a $y$ es
\begin{equation*}
\frac{\partial^2 T}{\partial y^2} \approx \frac{T_{i,j+1}^{l} - 2T_{i,j}^{l} + T_{i,j-1}^{l}}{(\Delta y)^2}
\end{equation*}
donde tanto para la segunda derivada en $x$ como en $y$ tenemos que los $\Delta$ son los pasos que se toman en cada coordenada. Podemos notar que el índice que está sumando en la derivada con respecto a $x$ es el $i$, mientras que cuando la derivada es con respecto a $y$, sumamos en $j$. Ambas fórmulas tienen un error de $o\left[(\Delta x)^2\right]$ y $o\left[(\Delta y)^2\right]$ respectivamente, lo cual significa que a medida que el tamaño del paso espacial se reduce, el error disminuye al cuadrado de este tamaño.

Sustituyendo estas aproximaciones en la ecuación de calor tenemos

\begin{equation}
\frac{T_{i,j}^{l+1} - T_{i,j}^{l}}{\Delta t} = \alpha\left( \frac{T_{i+1,j}^{l} - 2T_{i,j}^{l} + T_{i-1,j}^{l}}{(\Delta x)^2} + \frac{T_{i,j+1}^{l} - 2T_{i,j}^{l} + T_{i,j-1}^{l}}{(\Delta y)^2} \right)
\end{equation}

Ahora despejamos entonces el término $T_{i,j}^{l+1}$, que es la temperatura en el paso de tiempo siguiente (denotado por el superíndice $l+1$) y así obtenemos

\begin{equation*}
T_{i,j}^{l+1} = T_{i,j}^{l} + \alpha \frac{\Delta t}{(\Delta x)^2} \left( T_{i+1,j}^{l} - 2T_{i,j}^{l} + T_{i-1,j}^{l} \right) + \alpha \frac{\Delta t}{(\Delta y)^2} \left( T_{i,j+1}^{l} - 2T_{i,j}^{l} + T_{i,j-1}^{l} \right)
\end{equation*}

### Solución en Python

Primero implementamos el código en Python, el cuál está ampliamente comentado para saber que hacen las líneas de código.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import time

Para nuestro caso, vamos a tomar la placa en dos dimensiones como un cuadrado. Con esta consideración podemos definir ciertas cosas que facilitan la compresión e implementación del método numérico. Si lo tomamos como un cuadrado, tenemos que $\Delta x = \Delta y$ y así podemos definir entonces el término $T^{l+1}_{i,j}$ como 
\begin{align*}
T_{i,j}^{l+1} = T_{i,j}^{l} + \alpha \frac{\Delta t}{(\Delta x)^2} \left( T_{i+1,j}^{l} - 2T_{i,j}^{l} + T_{i-1,j}^{l} \right) + \alpha \frac{\Delta t}{(\Delta x)^2} \left( T_{i,j+1}^{l} - 2T_{i,j}^{l} + T_{i,j-1}^{l} \right) = T_{i,j}^{l} + \alpha \frac{\Delta t}{(\Delta x)^2} \left(T_{i+1,j}^{l} + T_{i-1,j}^{l} + T_{i,j+1}^{l} + T_{i,j-1}^{l} - 4T_{i,j}^{l} \right)
\end{align*}

Vamos entonces a definir como una variable nueva llamada $\gamma$ el factor que depende tanto de la constante de difusión como del paso temporal y el paso espacial. Por lo tanto $$\gamma = \alpha \dfrac{\Delta t}{(\Delta x)^2}$$ y así escribimos que 
\begin{align*}
T_{i,j}^{l+1}  = T_{i,j}^{l} + \gamma \left(T_{i+1,j}^{l} + T_{i-1,j}^{l} + T_{i,j+1}^{l} + T_{i,j-1}^{l} - 4T_{i,j}^{l} \right)
\end{align*}

Hay un factor muy importante a tomar en cuenta y es la estabilidad del método para ecuaciones diferenciales de este tipo. En el contexto de la ecuación de difusión (como la ecuación del calor), la condición de Courant garantiza la estabilidad de los métodos de diferencias finitas para la resolución numérica de ecuaciones diferenciales parciales. Para la ecuación de difusión unidimensional, el criterio de estabilidad viene dado por $$\Delta t \leq \dfrac{(\Delta x)^2}{2 \alpha}$$ sin embargo, usamos una condición aún más restrictiva cambiando el 2 por un 4 para asegurar la estabilidad numérica de la solución y no obtener respuestas sin sentido. Con esto seguimos entonces definiendo en Python más variables.

Definimos el tamaño de la cuadrícula de una dimensión de 50x50, en donde se calculará la distribución de temperatura a lo largo del tiempo y también definimos el número de pasos temporales para ver la evolución de la misma.

In [ ]:
inicio = time.time()
print("2D heat equation solver")

largo = 50   #dimensión de la placa (también es su ancho)
iteraciones = 1000   #iteraciones en el tiempo

alpha = 2   #número arbitrario para la constante de difusión, podría ser cualquiera
delta_x = 1   #paso espacial 
delta_t = (delta_x ** 2)/(4 * alpha)
gamma = (alpha * delta_t) / (delta_x ** 2)

Tenemos entonces una matriz tridimensional para almacenar la temperatura en cada punto de la malla para cada paso temporal. Además de eso damos un valor inicial de temperatura en todos los puntos de la malla aparte de definir las temperaturas que queremos en cada uno de los extremos de la placa. 

In [ ]:
T = np.empty((iteraciones, largo, largo))   #iniciar la solución de la malla para  T(l, i, j)

T_inicial = 0   #condición inicial de temperatura 

#condiciones de frontera 
T_sup = 100.0   #superior
T_izq = 50.0   #izquiera
T_inf = 25.0   #inferior
T_der = 0.0   #derecha

T.fill(T_inicial)   #llenamos toda la matriz T con el valor inicial 0

# establecemos la condiciones de frontera
T[:, (largo-1):, :] = T_sup
T[:, :, :1] = T_izq
T[:, :1, 1:] = T_inf
T[:, :, (largo-1):] = T_der

Se están utilizando arrays multidimensionales de NumPy para asignar valores a subconjuntos específicos de la matriz tridimensional $T$.
`T` es un array tridimensional de NumPy (np.ndarray) de dimensiones (iteraciones, largo, largo). En NumPy, el operador de `slicing (:)` se utiliza para seleccionar subconjuntos de un array. La lógica para la definición de las condiciones de frontera es la siguiente

- `:` en la primera dimensión selecciona todos los pasos temporales, `(largo-1)` en la segunda dimensión selecciona la última fila de la cuadrícula (borde superior) y `:` en la tercera dimensión selecciona todas las columnas en esa fila.
- `:1` en la tercera dimensión selecciona la primera columna de la cuadrícula (borde izquierdo).
- `:1` en la segunda dimensión selecciona la primera fila de la cuadrícula (borde inferior) y `1` en la tercera dimensión selecciona todas las columnas excepto la primera (para evitar asignar dos veces el punto (0,0) que ya se establece con `T_izq`).
- `(largo-1)` en la tercera dimensión selecciona la última columna de la cuadrícula (borde derecho).

Con todas las condiciones de frontera listas, ya podemos finalmente calcular el término $T^{l+1}_{i,j}$, el cual usa los valores anteriores de $T$ así como distintos valores de la temperatura para pasos en las diferentes coordenadas. 

In [ ]:
def calculate(T):
    for l in range(0, iteraciones-1, 1):
        for i in range(1, largo-1, delta_x):
            for j in range(1, largo-1, delta_x):
                T[l + 1, i, j] = T[l][i][j] + gamma * (T[l][i+1][j] + T[l][i-1][j] + T[l][i][j+1] + T[l][i][j-1] - 4*T[l][i][j])

    return T

fin = time.time()
print(fin-inicio)

Solo nos queda cambiar las condiciones iniciales para la temperatura interna de la placa como la temperatura en cada una de las fronteras y observar el comportamiento. Con las condiciones iniciales y de frontera deseadas, procedemos entonces con la graficación del resultado para una mayor comprensión. Con la siguiente función dibujamos un mapa de colores de la temperatura en la cuadrícula para el paso temporal $l$. Procuramos limpiar la figura actual de Matplotlib, eliminando cualquier contenido previo (gráficos, etiquetas, etc.), lo cual nos garantiza que cada nueva llamada a la función comience con un gráfico vacío, evitando superposiciones con gráficos anteriores.

In [ ]:
def mapa(T_l, l):

    plt.clf() #gráfico vacío

    plt.title(f"Temperatura en t = {l*delta_t:.3f} unidades temporales")
    plt.xlabel("x(t)")
    plt.ylabel("y(t)")

    plt.pcolormesh(T_l, cmap="hot", vmin=0, vmax=100) #esquema de colores
    plt.colorbar() #barra de colores

    return plt

# Calculamos la evolución de la temperatura en toda la cuadrícula.
T = temperatura(T)

def animate(l):
    mapa(T[l], l)

anim = animation.FuncAnimation(plt.figure(), animate, interval=10, frames=iteraciones, repeat=False)

anim.save("heat_equation_solution.mp4") #guardar

fin2 = time.time()
print(fin2-inicio)

En en título de gráfico tenemos que `delta_t` es el intervalo de tiempo entre pasos, mientras que `l*delta_t`calcula el tiempo real asociado al paso actual y `:.3f` formatea el tiempo para mostrar 3 decimales. Asímismo, lo que hicimos fue crear un mapa de colores basado en los valores de `T_l`, la matriz bidimensional de temperaturas en un paso temporal específico, donde `vmin=0, vmax=100` establece los límites de la escala de colores.

Posteriormente generamos una visualización para el paso temporal $l$ y tenemos que el parámetro `interval=10` declara el tiempo entre cuadros en milisegundos y el parámetro `frames=iteraciones` declara el número de cuadros, que es uno por cada iteración. La animación basicamente actualiza la cuadrícula en función del tiempo.

### Distintas condiciones iniciales

Con las primeras condiciones iniciales y de frontera, obtuvimos una animación que muestra como se distribuye la temperatura por la placa. Ahora cambiamos las condiciones y el código para conocer el comportamiento en distintas configuraciones. Iniciamos con la placa en $0^{\circ}$ y cada extremo a una temperatura distinta. Ahora ponemos la placa a $100^{\circ}$ y cada extremo a $0^{\circ}$ para visualizar la conducta.

Los cambios realizados al código son

In [ ]:
T_inicial = 100.0   #condición inicial de temperatura 

#condiciones de frontera 
T_sup = 0.0   #superior
T_izq = 0.0   #izquiera
T_inf = 0.0   #inferior
T_der = 0.0   #derecha

#T.fill(T_inicial)   #llenamos toda la matriz T con el valor inicial 0
T[0, :, :] = T_inicial

donde establecemos para el tiempo inicial (o primera iteración con `0`) la temperatura inicial y se le asigna a todas las filas y columnas con `[:, ;]`. 

En todo momento hemos utilizado condiciones de frontera estáticas, ya que la temperatura de los extremos de la placa es la misma en todo momento. No obstante, también se puede programar condiciones de frontera dinámicas y que la temperatura cambie en los bordes de alguna manera mientras transcurre el tiempo. Utilizamos entonces condiciones oscilantes periódicas con funciones trigonométricas. Tenemos que estas se definen como 
\begin{equation*}
    T_\text{frontera}(l)= T_\text{random}+ 50\sin\left( \frac{2\pi l}{100}\right)
\end{equation*}
donde $50$ representa una amplitud y $100$ es el periodo, que en este caso es la cantidad de iteraciones para que se complete un ciclo. Si el periodo es $100$, cada $100$ iteraciones las temperaturas vuelven a su punto de partida. Este cambio en las condiciones de frontera supone un cambio en el código.

In [ ]:
T_inicial = 10.0
T.fill(T_inicial)

def temperatura(T):

    for l in range(0, iteraciones-1, 1):
        # condiciones de frontera dinámicas en cada iteración
        T[l, 0, :] = 25 + 50 * np.sin(2 * np.pi * l / 100)     # superior
        T[l, -1, :] = 0 + 50 * np.sin(2 * np.pi * l / 100)     # inferior
        T[l, :, 0] = 50 + 50 * np.sin(2 * np.pi * l / 100)     # izquierda
        T[l, :, -1] = 75 + 50 * np.sin(2 * np.pi * l / 100)    # derecha

    for l in range(0, iteraciones-1, 1):
        for i in range(1, largo-1, delta_x):
            for j in range(1, largo-1, delta_x):
                T[l + 1, i, j] = T[l][i][j] + gamma * (T[l][i+1][j] + T[l][i-1][j] + T[l][i][j+1] + T[l][i][j-1] - 4*T[l][i][j])


    return T

Anidamos otro `for`en la función de temperatura para que también corra con las iteraciones. Tenemos entonces que las temperaturas suban y bajan de manera periódica entre $T_\text{random} \pm A$. Aparte de eso, podemos cambiar las funciones trigonométricas. Para otro set de condiciones dinámicas tenemos que 

In [ ]:
        T[l, 0, :] = 0 + 50 * np.sin(2 * np.pi * l / 100)      # superior
        T[l, -1, :] = 50 + 50 * np.cos(2 * np.pi * l / 100)    # inferior
        T[l, :, 0] = 25 + 50 * np.cos(2 * np.pi * l / 100)     # izquierda
        T[l, :, -1] = 75 + 50 * np.sin(2 * np.pi * l / 100)    # derecha

Se puede entonces jugar con la temperatura aleatoria inicial que queremos, la amplitud asociada a la función trigonométrica y el periodo de oscilación, dando resulta a distintas combinaciones que producen distintas animaciones.